<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/LiH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I'll try to simulate He2 and find the ground state energy of the molecule

In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 33.8 MB/s eta 0:00:00


In [2]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

symbols = ["Li", "H"]
geometry = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 2.969280527]])
molecule = qchem.Molecule(symbols, geometry)

In [3]:
H, qubits = qchem.molecular_hamiltonian(
    molecule,
    active_electrons=2,
    active_orbitals=5
)

active_electrons = 2

singles, doubles = qchem.excitations(active_electrons, qubits)

print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 24


Note that we have a total of 24 excitations which can be represented by the same number of excitation gates 4. Let’s now use an AdaptiveOptimizer implemented in PennyLane to construct an adaptive circuit.

In [4]:
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations

We now define an initial circuit that prepares a Hartree-Fock state and returns the expectation value of the Hamiltonian. We also need to define a device.

In [5]:
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.expval(H)

We instantiate the optimizer and use it to build the circuit adaptively.

In [ ]:
opt = qml.optimize.AdaptiveOptimizer()
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
    if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        print(qml.draw(circuit, decimals=None)())
        print()
    if gradient < 3e-3:
        break

n = 0,  E = -7.86266588 H, Largest Gradient = 0.124
0: ──X─╭G²─┤
1: ──X─├G²─┤
2: ────│───┤
3: ────│───┤
4: ────│───┤
5: ────│───┤
6: ────│───┤
7: ────│───┤
8: ────├G²─┤
9: ────╰G²─┤

  ╭<(-5.727445042514035*I)+(-0.29921779790521325*Z)+(0.003964035094958761*Y@Z@Y)+(0.003964035094958761*X@Z@X)+(0.011179687382654513*Y@Z@Z@Z@Z@Z@Z@Z@Y)+(0.011179687382654513*X@Z@Z@Z@Z@Z@Z@Z@X)+(-0.39038057899812817*Z)+(0.052915132228249076*Z@Z)+(0.023938079372329082*Y@Z@Z@Z@Z@Z@Y)+(0.023938079372329082*X@Z@Z@Z@Z@Z@X)+(0.004239839868248589*Z@Y@Z@Z@Z@Z@Z@Y)+(0.004239839868248589*Z@X@Z@Z@Z@Z@Z@X)+(-0.4257208570451667*Z)+(0.062002444193028125*Z@Z)+(-0.42572085704533724*Z)+(0.062002444193056824*Z@Z)+(-0.567800972062882*Z)+(0.08283156204785036*Z@Z)+(-0.2992177979052133*Z)+(0.12232658337439485*Z@Z)+(-0.012024224161438734*Y@Y)+(-0.012024224161438734*X@X)+(0.032073745988023046*Y@Z@Z@Z@Z@Z@Z@Y)+(0.032073745988023046*X@Z@Z@Z@Z@Z@Z@X)+(0.003964035094958764*Y@Z@Y)+(-0.012024224161438734*Z@Y@Z@Y)+(0.003964035094958764*X@

The resulting energy matches the exact energy of the ground electronic state of LiH, which is -7.8825378193 Ha, within chemical accuracy.

In [ ]:
@qml.qnode(dev)
def circuit():
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.expval(H)

opt = qml.optimize.AdaptiveOptimizer()
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool, drain_pool=True)
    if i % 2 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        print(qml.draw(circuit, decimals=None)())
        print()
    if gradient < 3e-3:
        break

[1]
A. Peruzzo, J. McClean et al., “A variational eigenvalue solver on a photonic quantum processor”. Nat. Commun. 5, 4213 (2014).

[2]
Y. Cao, J. Romero, et al., “Quantum Chemistry in the Age of Quantum Computing”. Chem. Rev. 2019, 119, 19, 10856-10915.


Adaptive circuits for quantum chemistry
https://pennylane.ai/qml/demos/tutorial_adaptive_circuits/